# DData Extraction
This notebook uses the Domain developer API to extract data and create various datasets

In [18]:
import pandas as pd
import requests
import json
PROJECT_PATH = '/Users/elmi/Projects/HouseHunt'

headers = {
    'X-Api-Key': 'key_aa83d5090fa4db8cd2ddbad271af541f'
}

state = 'QLD'

## Suburb Data

In [ ]:
# Read raw addressLocators API data
suburbs = pd.read_csv(f"{PROJECT_PATH}/data/raw/20180105__brisbane_190_suburbs_list_.csv", header=None)[0].to_list()

api_response = []

for suburb in suburbs:
    r = requests.get(f'https://api.domain.com.au/v1/addressLocators?searchLevel=Suburb&suburb={suburb}&state={state}', headers=headers)
    if not r.ok:
        print(f'Skipping {suburb}\n {r.text}')
    api_response.extend(json.loads(r.text))

with open(f'{PROJECT_PATH}/data/raw/api_addressLocators.json', 'w') as f:
    json.dump(api_response, f)

api_response[0].keys()

In [92]:
suburbs = pd.read_csv(f"{PROJECT_PATH}/data/raw/20180105__brisbane_190_suburbs_list_.csv", header=None)[0].to_list()
cached_suburbs = pd.read_csv(f"{PROJECT_PATH}/data/processed/suburbs.csv")['Suburb'].to_list()

set(suburbs) - set(cached_suburbs)

{'Mt Coot-tha',
 'Mt Crosby',
 'Mt Gravatt',
 'Mt Gravatt East',
 'Mt Ommaney',
 'Port of Brisbane',
 'Upper Mt Gravatt'}

In [22]:
# Process id, suburb, postcode and state
api_response = json.load(open(f'{PROJECT_PATH}/data/raw/api_addressLocators.json'))
api_processed = []
for r in api_response:
    out = {}
    for c in r['addressComponents']:
        out[c['component']] = c['shortName']
    out['suburb_id'] = r['ids'][0]['id']
    if len(r['ids']) != 1:
        print(f"Multiple IDs found for {out['Suburb']}")
    api_processed.append(out)

pd.DataFrame(api_processed).to_csv(f"{PROJECT_PATH}/data/processed/suburbs.csv", index=False)
api_processed[:5]  

pd.DataFrame(api_processed).to_csv(f"{PROJECT_PATH}/data/processed/suburbs.csv", index=False)

## Suburb Performance Statistics 

## *** RUN FROM HERE TOMOROW ***

In [100]:
# Read raw suburbPerformanceStatistics API data
suburb_ids = pd.read_csv(f"{PROJECT_PATH}/data/processed/suburbs.csv")['suburb_id'].to_list()

property_category = 'unit'
span=6
t_from = 1
t_to = 10

api_response = []
for suburb_id in suburb_ids:
    r = requests.get(f'https://api.domain.com.au/v1/suburbPerformanceStatistics?state={state}&suburbId={suburb_id}&propertyCategory={property_category}&chronologicalSpan={span}&tPlusFrom={t_from}&tPlusTo={t_to}', headers=headers)
    if not r.ok or not r.text:
        print(f'Skipping {suburb_id}: {r.text}')
        continue
    r_dict = json.loads(r.text)
    r_dict['suburb_id'] = suburb_id
    api_response.append(r_dict)

with open(f'{PROJECT_PATH}/data/raw/api_suburbPerformanceStatistics.json', 'w') as f:
    json.dump(api_response, f)

api_response[0].keys()

Skipping 104: {
  "type": "https://developer.domain.com.au/docs/latest/conventions/rate-limiting",
  "title": "Quota Exceeded",
  "detail": "Exceeded rate limit for current period"
}
Skipping 324: {
  "type": "https://developer.domain.com.au/docs/latest/conventions/rate-limiting",
  "title": "Quota Exceeded",
  "detail": "Exceeded rate limit for current period"
}
Skipping 344: {
  "type": "https://developer.domain.com.au/docs/latest/conventions/rate-limiting",
  "title": "Quota Exceeded",
  "detail": "Exceeded rate limit for current period"
}
Skipping 364: {
  "type": "https://developer.domain.com.au/docs/latest/conventions/rate-limiting",
  "title": "Quota Exceeded",
  "detail": "Exceeded rate limit for current period"
}
Skipping 464: {
  "type": "https://developer.domain.com.au/docs/latest/conventions/rate-limiting",
  "title": "Quota Exceeded",
  "detail": "Exceeded rate limit for current period"
}
Skipping 914: {
  "type": "https://developer.domain.com.au/docs/latest/conventions/ra

IndexError: list index out of range

In [ ]:
# cached_stats = json.load(open(f'{PROJECT_PATH}/data/raw/api_suburbPerformanceStatistics_v2.json'))

# out = []
# for x in cached_stats:
#     suburb_id = list(x.keys())[0]
#     value = x[suburb_id]
#     value['suburb_id'] = suburb_id
#     out.append(value)

# with open(f'{PROJECT_PATH}/data/raw/api_suburbPerformanceStatistics.json', 'w') as f:
#     json.dump(out, f)

In [99]:
# Process suburb_id, property_category, year, month, stats
api_response = json.load(open(f'{PROJECT_PATH}/data/raw/api_suburbPerformanceStatistics.json'))
api_processed = []
for r in api_response:
    suburb_id = r['suburb_id']
    suburb = r['header']['suburb']
    property_category = r['header']['propertyCategory']
    out = []
    for s in r['series']['seriesInfo']:
        row = {'suburb_id': suburb_id, 'suburb': suburb, 'property_category': property_category, 'year': s['year'], 'month': s['month']} | s['values']
        out.append(row)
    api_processed.extend(out)

pd.DataFrame(api_processed).to_csv(f"{PROJECT_PATH}/data/processed/suburb_performance_stats.csv", index=False)

api_processed[0].keys()  

dict_keys(['suburb_id', 'suburb', 'property_category', 'year', 'month', 'medianSoldPrice', 'numberSold', 'highestSoldPrice', 'lowestSoldPrice', '5thPercentileSoldPrice', '25thPercentileSoldPrice', '75thPercentileSoldPrice', '95thPercentileSoldPrice', 'medianSaleListingPrice', 'numberSaleListing', 'highestSaleListingPrice', 'lowestSaleListingPrice', 'auctionNumberAuctioned', 'auctionNumberSold', 'auctionNumberWithdrawn', 'daysOnMarket', 'discountPercentage', 'medianRentListingPrice', 'numberRentListing', 'highestRentListingPrice', 'lowestRentListingPrice'])

curl -X GET "https://api.domain.com.au/v1/suburbPerformanceStatistics?state=QLD&suburbId=104&propertyCategory=unit&chronologicalSpan=3&tPlusFrom=1&tPlusTo=20" -H "accept: application/json" -H "X-Api-Key: key_aa83d5090fa4db8cd2ddbad271af541f"